In [1]:
# -*- coding: utf-8 -*-
"""Untitled73.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1ci00gmjMZKlLiXsS106pHQiM_Yt6kzHt
"""

# -*- coding: utf-8 -*-
"""CodeFinal.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1b8FnEoegk3itYEwv8_yuBpEF3uCAv4Ta
"""

# -*- coding: utf-8 -*-
"""Untitled23.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1gyqSXuq-Wga83Gkhpaox72ljcg00I8WQ
"""

import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from urllib.parse import urljoin
import time
from collections import defaultdict
import math  # For log in IDF
import pandas as pd
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np
from urllib.parse import urlparse, urljoin

class SearchEngine:
    def __init__(self):
        """
        Initializes the SearchEngine with a PorterStemmer, a set of stop words,
        and internal mappings/data structures for indexing.

        Attributes:
            stemmer (PorterStemmer): Stemmer used for normalizing words.
            stop_words (set): Set of English stopwords to exclude from indexing.
            url_to_id (dict): Maps a URL string to a unique integer ID.
            id_to_url (dict): Maps a unique integer ID back to its URL string.
            next_id (int): Next available integer ID for a new URL.
            inverted_index (defaultdict): Maps a stemmed word to a dict of {doc_id: count}.
            global_index (defaultdict): Similar to inverted_index but stores {word: {url: count}}.
        """
        self.stemmer = PorterStemmer()
        self.stop_words = {
            'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any',
            'are', 'aren', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between',
            'both', 'but', 'by', 'can', 'could', 'did', 'do', 'does', 'doing', 'don', 'down', 'during',
            'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her',
            'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is',
            'it', 'its', 'itself', 'just', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not',
            'now', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out',
            'over', 'own', 's', 'same', 'she', 'should', 'so', 'some', 'such', 'than', 'that', 'the',
            'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
            'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', 'we', 'were', 'what', 'when',
            'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'you', 'your', 'yours',
            'yourself', 'yourselves','et','wh','la','à','last','also','include','chang','includ','thi'
        }

        # Add domain-specific terms
        self.healthcare_terms = {
            'health', 'medical', 'doctor', 'patient', 'hospital',
            'treatment', 'care', 'disease', 'medicine', 'therapy',
            'policy', 'insurance', 'healthcare', 'clinic', 'chronic',
            'diagnosis', 'prevention', 'wellness', 'specialist'
        }

        self.url_to_id = {}
        self.id_to_url = {}
        self.next_id = 1

        # inverted_index[word][doc_id] = count of word in doc
        self.inverted_index = defaultdict(lambda: defaultdict(int))

        # global_index[word][url] = count of word in url
        self.global_index = defaultdict(lambda: defaultdict(int))

    def is_valid_word(self, word):
        """
        Validates if a word is acceptable for indexing
        """
        # Filter out strings that contain any digits
        if any(char.isdigit() for char in word):
            return False

        # Minimum length check
        if len(word) < 3:
            return False

        # Must contain only English letters
        if not all(ord('a') <= ord(c.lower()) <= ord('z') for c in word if c.isalpha()):
            return False

        # Must be only alphabetic characters (no special chars)
        if not word.isalpha():
            return False

        return True

    def get_links_hits(self, soup, base_url):
      """
      Extracts all <a> tag hyperlinks from a BeautifulSoup page and converts them to absolute URLs.

      Args:
          soup (BeautifulSoup): The parsed HTML content.
          base_url (str): The base URL to resolve relative links.

      Returns:
          set: A set of absolute URLs found in the page.
      """
      links = set()
      if soup:
          for link in soup.find_all('a', href=True):
              url = link['href']
              absolute_url = urljoin(base_url, url)
              if absolute_url.startswith(base_url):
                  links.add(absolute_url)
      return links

    def analyze_link_structure(self, max_iterations=20, epsilon=1e-8):
        """
        Performs the HITS algorithm to compute hub and authority scores
        for the URLs that the search engine has crawled.

        Args:
            max_iterations (int): Maximum number of iterations for the HITS algorithm.
            epsilon (float): Convergence threshold for stopping early.

        Returns:
            dict: A dictionary containing:
                - 'hub_scores' (dict): Hub scores for each URL ID.
                - 'auth_scores' (dict): Authority scores for each URL ID.
                - 'outgoing_links' (defaultdict): Mapping from URL ID to a set of outgoing link IDs.
                - 'incoming_links' (defaultdict): Mapping from URL ID to a set of incoming link IDs.
        """
        outgoing_links = defaultdict(set)
        incoming_links = defaultdict(set)

        # # Build the adjacency matrix
        # for url_id in self.id_to_url:
        #     url = self.id_to_url[url_id]
        #     soup = self.fetch_page(url)
        #     if soup:
        #         links = self.get_links_hits(soup, url)
        #         for link in links:
        #             if link in self.url_to_id:
        #                 target_id = self.url_to_id[link]
        #                 outgoing_links[url_id].add(target_id)
        #                 incoming_links[target_id].add(url_id)

        #Build initial link structure
        for url_id in self.id_to_url:
            url = self.id_to_url[url_id]
            soup = self.fetch_page(url)
            if soup:
                links = set()
                for link in soup.find_all('a', href=True):
                    absolute_url = urljoin(url, link['href'])
                    if absolute_url in self.url_to_id:
                        target_id = self.url_to_id[absolute_url]
                        links.add(target_id)
                if links:  # Only include pages that have outgoing links
                    outgoing_links[url_id] = links
                    # Update incoming links
                    for target_id in links:
                        incoming_links[target_id].add(url_id)

        # Identify active pages (those with links)
        active_pages = set(outgoing_links.keys()) | set(incoming_links.keys())

        # Initialize hub and authority scores only for active pages
        hub_scores = {url_id: 1.0 for url_id in active_pages}
        auth_scores = {url_id: 1.0 for url_id in active_pages}

        # HITS algorithm iterations
        EPSILON = 1e-10  # Minimum threshold for normalization
        for iteration in range(max_iterations):
            old_hub_scores = hub_scores.copy()
            old_auth_scores = auth_scores.copy()

            # Update authority scores
            new_auth = {}
            for page in active_pages:
                new_auth[page] = sum(hub_scores[incoming] for incoming in incoming_links[page])

            # Normalize authority scores
            auth_norm = math.sqrt(sum(score * score for score in new_auth.values()))
            if auth_norm < EPSILON:
                auth_norm = EPSILON
            for page in new_auth:
                new_auth[page] /= auth_norm
            auth_scores = new_auth

            # Update hub scores
            new_hub = {}
            for page in active_pages:
                new_hub[page] = sum(auth_scores[outgoing] for outgoing in outgoing_links[page])

            # Normalize hub scores
            hub_norm = math.sqrt(sum(score * score for score in new_hub.values()))
            if hub_norm < EPSILON:
                hub_norm = EPSILON
            for page in new_hub:
                new_hub[page] /= hub_norm
            hub_scores = new_hub

            # Check convergence using maximum difference
            max_diff = max(
                max(abs(auth_scores[p] - old_auth_scores[p]) for p in active_pages),
                max(abs(hub_scores[p] - old_hub_scores[p]) for p in active_pages)
            )

            if max_diff < epsilon:
                print(f"Converged after {iteration + 1} iterations with max difference: {max_diff:.6e}")
                break

        # Print analysis results
        print("\n=== HITS Algorithm Analysis ===")
        print("\nTop Hubs (pages that point to good authorities):")
        sorted_hubs = sorted(hub_scores.items(), key=lambda x: x[1], reverse=True)
        for url_id, score in sorted_hubs[:5]:
            print(f"URL ID: {url_id}, URL: {self.id_to_url[url_id]}")
            print(f"Hub Score: {score:.6f}")
            print(f"Number of Outgoing Links: {len(outgoing_links[url_id])}")

        print("\nTop Authorities (pages pointed to by good hubs):")
        sorted_authorities = sorted(auth_scores.items(), key=lambda x: x[1], reverse=True)
        for url_id, score in sorted_authorities[:5]:
            print(f"URL ID: {url_id}, URL: {self.id_to_url[url_id]}")
            print(f"Authority Score: {score:.6f}")
            print(f"Number of Incoming Links: {len(incoming_links[url_id])}")

        return {
            'hub_scores': hub_scores,
            'auth_scores': auth_scores,
            'outgoing_links': outgoing_links,
            'incoming_links': incoming_links
        }

    def get_url_id(self, url):
        """
        Retrieves the unique integer ID for a given URL, assigning a new one if necessary.

        Args:
            url (str): The URL to look up or assign an ID to.

        Returns:
            int: The unique integer ID corresponding to the given URL.
        """
        if url not in self.url_to_id:
            self.url_to_id[url] = self.next_id
            self.id_to_url[self.next_id] = url
            self.next_id += 1
        return self.url_to_id[url]

    def fetch_page(self, url, max_retries=3):
        """
        Fetches page content with retry logic for 404 errors.

        Args:
            url (str): URL to fetch
            max_retries (int): Maximum retry attempts

        Returns:
            BeautifulSoup or None
        """
        retries = 0
        while retries < max_retries:
            try:
                time.sleep(1)
                response = requests.get(url, timeout=10)

                if response.status_code == 200:
                    return BeautifulSoup(response.text, 'html.parser')
                elif response.status_code == 404:
                    print(f"404 error for {url}, attempt {retries + 1}/{max_retries}")
                    retries += 1
                    if retries < max_retries:
                        time.sleep(2)  # Wait before retrying
                    continue
                else:
                    print(f"Failed to fetch {url}: Status code {response.status_code}")
                    return None

            except Exception as e:
                print(f"Error fetching {url}: {str(e)}")
                return None

        print(f"Failed to fetch {url} after {max_retries} attempts")
        return None

    def get_links(self, soup, base_url):
        """
        Extracts all <a> tag hyperlinks from a BeautifulSoup page and converts them to absolute URLs.

        Args:
            soup (BeautifulSoup): The parsed HTML content.
            base_url (str): The base URL to resolve relative links.

        Returns:
            set: A filtered set of absolute URLs found in the page.
        """
        links = set()
        if not soup:
            return links

        excluded_patterns = [
            '/contact',
            'contact-form',
            'kontakt',
            'impressum',
            'imprint',
            'privacy',
            'datenschutz',
            'disclaimer',
            'login',
            'register',
            'signin',
            'signup',
            'search',
            'suche',
            '/index',
            'print',
            'rss',
            'feed',
            'sitemap',
            'archive',
            'newsletter',
            'tasks-and-organisation',
            'user-information',
            'index.html',
            '#c',
            'release',
            'press-release',
            'article_list',
            'press-contact',
        ]

        excluded_extensions = [
            '.pdf', '.doc', '.docx', '.xls', '.xlsx',
            '.zip', '.rar', '.mp3', '.mp4', '.avi',
            '.jpg', '.jpeg', '.png', '.gif'
        ]

        for link in soup.find_all('a', href=True):
            url = link['href']
            absolute_url = urljoin(base_url, url)

            if absolute_url.startswith(base_url):
                # מחלץ את החלק שאחרי ה-base_url
                path = absolute_url[len(base_url):]
                # בודק אם החלק שאחרי ה-base_url מכיל תבניות אסורות
                if not any(pattern in path.lower() for pattern in excluded_patterns):
                    # בודק סיומות אסורות
                    if not any(absolute_url.lower().endswith(ext) for ext in excluded_extensions):
                        links.add(absolute_url)

        return links

    def index_words(self, soup, url):
        """
        Extracts and indexes the words from a BeautifulSoup page, updating the
        inverted and global indexes with stemmed words.

        Args:
            soup (BeautifulSoup): Parsed HTML content.
            url (str): The URL of the page being indexed.

        Returns:
            dict: A dictionary of {stemmed_word: count_in_this_document} for the page.
        """
        index = defaultdict(int)
        url_id = self.get_url_id(url)

        if soup:
            words = re.findall(r'\w+', soup.get_text())
            for word in words:
                word = word.lower()

                stemmed_word = self.stemmer.stem(word)

                # נבדוק אם מותר להשתמש במילה (לא מספר, לא קצר מדי וכד')
                if not self.is_valid_word(stemmed_word):
                    continue

                if stemmed_word not in self.stop_words:
                    if stemmed_word == "germani":
                      stemmed_word="german"

                    # Increase counters in the local index (for this page)
                    index[stemmed_word] += 1

                    # Update the global/inverted index
                    self.inverted_index[stemmed_word][url_id] += 1
                    self.global_index[stemmed_word][url] += 1

        return dict(index)

    def crawl_and_index(self, start_url, query, max_pages=51):
        """
        Performs focused crawling based on query relevance while maintaining soup cache for PageRank.

        Args:
            start_url (str): Initial URL
            query (str): Search query
            max_pages (int): Maximum pages to crawl

        Returns:
            tuple: (page_indexes, soup_cache)
        """
        query_words = re.findall(r'\w+', query.lower())
        query_terms = {self.stemmer.stem(word) for word in query_words
                      if word not in self.stop_words}

        #visited = set()
        #to_visit = {start_url}

        visited = set([start_url])  # Add start_url to visited immediately
        links = self.get_links(self.fetch_page(start_url), start_url)
        to_visit = set(links)  # Initialize with links from start page
        page_indexes = {}
        potential_pages = []
        seen_content = set()
        soup_cache = {}

        while to_visit and len(visited) < 100:
            url = to_visit.pop()
            if url in visited:
                continue

            print(f"Crawling: {url} (ID: {self.get_url_id(url)})")

            soup = self.fetch_page(url)
            if soup:
                # Cache the soup object
                soup_cache[url] = soup

                title = soup.find('title')
                headers = soup.find_all(['h1', 'h2', 'h3'])

                title_terms = set()
                if title:
                    title_words = re.findall(r'\w+', title.text.lower())
                    title_terms.update(self.stemmer.stem(w) for w in title_words
                                      if w not in self.stop_words)
                for h in headers:
                    header_words = re.findall(r'\w+', h.text.lower())
                    title_terms.update(self.stemmer.stem(w) for w in header_words
                                      if w not in self.stop_words)

                page_index = self.index_words(soup, url)
                page_terms = set(page_index.keys())

                # Always index the page for PageRank analysis
                doc_id = self.get_url_id(url)
                page_indexes[doc_id] = page_index

                # Calculate relevance score if query terms exist
                if page_terms & query_terms:
                    total_words = sum(page_index.values())
                    content = frozenset(page_index.items())

                    if content not in seen_content:
                        seen_content.add(content)
                        page_score = 0

                        for term in query_terms:
                            if term in page_index:
                                tf = page_index[term] / total_words
                                if term in title_terms:
                                    tf *= 1.5
                                df = len(self.inverted_index[term])
                                idf = math.log(len(self.url_to_id) / (df + 1))
                                page_score += tf * idf

                        norm = math.sqrt(page_score ** 2)
                        if norm > 0:
                            page_score /= norm

                        potential_pages.append((doc_id, page_index, page_score))

                visited.add(url)
                links = self.get_links(soup, start_url)
                to_visit.update(links - visited)

        # Sort pages by relevance score
        if potential_pages:
            top_pages = sorted(potential_pages, key=lambda x: x[2], reverse=True)[:50]
            print("\nTop 50 pages by relevance score:")
            for doc_id, page_index, score in top_pages:
                url_str = self.id_to_url.get(doc_id, "N/A")
                print(f"Page ID: {doc_id}, URL: {url_str}")

            relevant_indexes = {doc_id: index for doc_id, index, _ in top_pages}
            return relevant_indexes, soup_cache
        else:
            # אם לא נמצאו עמודים רלוונטיים – מיון על פי כמות התוכן (סכום ספירות המילים)
            sorted_page_indexes = dict(sorted(page_indexes.items(),
                                              key=lambda item: sum(item[1].values()),
                                              reverse=True))[:50]
            print("\nTop 50 pages by content size:")
            for doc_id, page_index in sorted_pages:
                url_str = self.id_to_url.get(doc_id, "N/A")
                total_words = sum(page_index.values())
                print(f"Page ID: {doc_id}, URL: {url_str}")

        return sorted_page_indexes, soup_cache

    def is_english_word(self, word):
      """
      בודק אם המילה מכילה רק תווים באנגלית
      """
      return all(ord('a') <= ord(c.lower()) <= ord('z') for c in word)

    def is_english_word(word):
        """
        Checks if a word contains only English characters.
        Returns True if the word only contains ASCII letters a-z or A-Z.
        """
        return all(ord('a') <= ord(c.lower()) <= ord('z') for c in word if c.isalpha())

    def compute_inverted_index(self, page_indexes):
        """
        מחשב inverted index עם נורמליזציית L2, כולל סינון מילות עצירה
        """

        # אימון מודל Word2Vec על המסמכים - סינון מילות עצירה כבר בשלב זה
        sentences = []
        for doc_id, word_counts in page_indexes.items():
            sentence = []
            for word, count in word_counts.items():
                # בדיקה שהמילה אינה מילת עצירה לפני הוספתה
                if word not in self.stop_words:
                    sentence.extend([word] * count)
            sentences.append(sentence)

        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

        # חישוב חשיבות המילים עם TF-IDF
        word_importance = defaultdict(float)
        N = len(page_indexes)  # מספר המסמכים

        # חישוב TF-IDF vectors לכל מסמך
        doc_vectors = {}
        for doc_id, word_counts in page_indexes.items():
            vector = defaultdict(float)
            # חישוב סך המילים ללא מילות עצירה
            total_words = sum(count for word, count in word_counts.items()
                            if word not in self.stop_words)

            for word, count in word_counts.items():
                # התעלמות ממילות עצירה
                if word not in self.stop_words and word in model.wv:
                    # חישוב TF
                    tf = count / total_words if total_words > 0 else 0

                    # חישוב IDF (רק עבור מסמכים עם המילה שאינה מילת עצירה)
                    df = len([1 for doc in page_indexes.values()
                            if word in doc and word not in self.stop_words])
                    idf = math.log(N / df) if df > 0 else 0

                    # TF-IDF score
                    vector[word] = tf * idf

            # L2 normalization
            magnitude = math.sqrt(sum(score * score for score in vector.values()))
            if magnitude > 0:
                for word in vector:
                    vector[word] /= magnitude

            doc_vectors[doc_id] = vector

        # חישוב חשיבות סופית של המילים
        for word in model.wv.index_to_key:
            if word not in self.stop_words:  # בדיקה נוספת למילות עצירה
                word_score = 0
                for doc_id, vector in doc_vectors.items():
                    if word in vector:
                        # הוספת ציון TF-IDF המנורמל
                        word_score += vector[word]

                        # חישוב דמיון סמנטי
                        try:
                            similar_words = model.wv.most_similar(word, topn=5)
                            # סינון מילים דומות שהן מילות עצירה
                            similar_words = [(w, s) for w, s in similar_words
                                          if w not in self.stop_words]
                            if similar_words:
                                semantic_score = sum(score for _, score in similar_words) / len(similar_words)
                                word_score *= (1 + semantic_score)
                        except KeyError:
                            continue

                word_importance[word] = word_score

        # בחירת 15 המילים המשמעותיות ביותר (שאינן מילות עצירה)
        top_15_words = sorted(word_importance.items(), key=lambda x: x[1], reverse=True)[:15]
        top_15_set = {word for word, _ in top_15_words}

        # יצירת inverted index חדש רק עם 15 המילים הנבחרות
        new_inverted_index = defaultdict(lambda: defaultdict(int))
        for word in top_15_set:
            for doc_id, vector in doc_vectors.items():
                if word in vector:
                    new_inverted_index[word][doc_id] = vector[word]  # שמירת הערך המנורמל

        return new_inverted_index, top_15_words

    def save_to_excel(self, page_indexes, filename="results.xlsx"):
        """
        שומרת את תוצאות הניתוח לקובץ Excel
        """
        # שימוש ב-compute_inverted_index לקבלת ה-inverted index המעודכן וה-model
        new_inverted_index, top_15_words = self.compute_inverted_index(page_indexes)
        self.inverted_index = new_inverted_index

        # יצירת מודל Word2Vec חדש (נחוץ לחישוב מילים דומות)
        sentences = []
        for doc_id, word_counts in page_indexes.items():
            sentence = []
            for word, count in word_counts.items():
                sentence.extend([word] * count)
            sentences.append(sentence)

        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

        # יצירת DataFrame למיפוי URL-ID
        url_mappings = [{"URL ID": url_id, "URL": url}
                        for url, url_id in self.url_to_id.items()]
        url_df = pd.DataFrame(url_mappings)

        # יצירת DataFrame לספירות מילים בכל דף
        word_count_rows = []
        for url_id, index in page_indexes.items():
            for word, count in index.items():
                word_count_rows.append({
                    "URL ID": url_id,
                    "Word": word,
                    "Count": count
                })
        counts_df = pd.DataFrame(word_count_rows)

        # יצירת DataFrame ל-inverted index
        inverted_rows = []
        for word, importance in top_15_words:
            doc_counts = self.inverted_index[word]
            document_ids = ", ".join(map(str, sorted(doc_counts.keys())))
            doc_count = len(doc_counts)
            total_count = sum(doc_counts.values())
            avg_count = total_count / doc_count if doc_count > 0 else 0

            # חישוב מילים דומות באמצעות Word2Vec
            similar_words = "N/A"
            if word in model.wv:
                try:
                    similar = model.wv.most_similar(word, topn=3)
                    similar_words = ", ".join(f"{w}({s:.2f})" for w, s in similar)
                except KeyError:
                    pass

            inverted_rows.append({
                "Word": word,
                "Document IDs": document_ids,
                "Number of Documents": doc_count,
                "Total Occurrences": total_count,
                "Average Count per Document": f"{avg_count:.2f}",
                "Word2Vec Score": f"{importance:.4f}",
                "Similar Words": similar_words
            })

        inverted_df = pd.DataFrame(inverted_rows)

        # שמירה לקובץ Excel
        with pd.ExcelWriter(filename, engine="openpyxl") as writer:
            url_df.to_excel(writer, index=False, sheet_name="URL Mappings")
            counts_df = counts_df.sort_values(by="URL ID")
            counts_df.reset_index(drop=True, inplace=True)
            counts_df.to_excel(writer, index=False, sheet_name="Page Word Counts")
            inverted_df.to_excel(writer, index=False, sheet_name="Top 15 Inverted Index")

        # הדפסת סיכום
        print(f"\nResults saved to '{filename}'.")
        print("\nTop 15 Words with Word2Vec Analysis:")
        print("-" * 60)
        for word, importance in top_15_words:
            doc_count = len(self.inverted_index[word])
            total_count = sum(self.inverted_index[word].values())
            print(f"Word: {word}")
            print(f"- Importance Score: {importance:.4f}")
            print(f"- Appears in {doc_count} documents")
            print(f"- Total occurrences: {total_count}")
            print("-" * 30)

    def calculate_tf_idf_for_query(self, query, page_indexes, filename="results.xlsx"):
        """
        Calculates TF-IDF scores for each query word in each document, applies L2 normalization,
        and saves the results to an Excel sheet.

        Steps:
        1) Tokenize, remove stopwords, and stem the query.
        2) For each document, compute TF (term frequency), IDF (inverse document frequency),
          and TF-IDF for each query word.
        3) Apply L2 normalization to TF-IDF scores.
        4) Save results in a new sheet named "Query TF-IDF".

        Args:
            query (str): The user query.
            page_indexes (dict): Dictionary of {doc_id: {word: count}} representing indexed documents.
            filename (str): Name of the Excel file to append the TF-IDF results.

        Returns:
            None
        """
        # 1) Preprocess the query
        query_words = re.findall(r'\w+', query.lower())
        processed_query = []
        for qw in query_words:
            if qw not in self.stop_words:
                stemmed_qw = self.stemmer.stem(qw)
                processed_query.append(stemmed_qw)

        # 2) Collect stats for TF-IDF
        N = len(page_indexes)  # total number of documents
        doc_total_words = {}
        for doc_id, index_dict in page_indexes.items():
            total_count = sum(index_dict.values())
            doc_total_words[doc_id] = total_count

        # 3) Compute TF, IDF, TF-IDF (before normalization)
        # First, collect all TF-IDF values grouped by document
        doc_tf_idfs = defaultdict(list)  # {doc_id: [tf_idf values]}
        tf_idf_data = []  # Store all the data for later use

        for doc_id, index_dict in page_indexes.items():
            url = self.id_to_url[doc_id]
            total_words_in_doc = doc_total_words[doc_id]

            for qw in processed_query:
                # Calculate TF
                freq_in_doc = self.inverted_index[qw].get(doc_id, 0)
                tf = freq_in_doc / total_words_in_doc if total_words_in_doc > 0 else 0.0

                # Calculate IDF
                df = len(self.inverted_index[qw].keys())  # number of docs containing qw
                idf = math.log(N / df, 10) if df > 0 else 0.0
                if df > 0:
                   raw_idf = math.log(N / df, 10)
                   idf = max(raw_idf, 0)
                else:
                   idf = 0

                # Calculate TF-IDF
                tf_idf = tf * idf

                # Store TF-IDF for later normalization
                doc_tf_idfs[doc_id].append(tf_idf)

                # Store all data for final output
                tf_idf_data.append({
                    "doc_id": doc_id,
                    "url": url,
                    "query_word": qw,
                    "tf": tf,
                    "idf": idf,
                    "tf_idf": tf_idf
                })

        # 4) Compute L2 norms for each document
        doc_norms = {}
        for doc_id, tf_idfs in doc_tf_idfs.items():
            l2_norm = math.sqrt(sum(x * x for x in tf_idfs))
            doc_norms[doc_id] = l2_norm if l2_norm > 0 else 1.0  # Avoid division by zero

        # 5) Create final results with normalized values
        tf_idf_rows = []
        for data in tf_idf_data:
            doc_id = data["doc_id"]
            # Apply L2 normalization
            normalized_tf_idf = data["tf_idf"] / doc_norms[doc_id]

            tf_idf_rows.append({
                "Document ID": doc_id,
                "URL": data["url"],
                "Query Concept": data["query_word"],
                "TF": data["tf"],
                "IDF": data["idf"],
                "TF-IDF": data["tf_idf"],
                "Normalized TF-IDF": normalized_tf_idf
            })

        # 6) Create DataFrame and save to Excel
        tf_idf_df = pd.DataFrame(tf_idf_rows)

        # Sort by Document ID and Normalized TF-IDF for better readability
        tf_idf_df = tf_idf_df.sort_values(
            by=['Document ID', 'Normalized TF-IDF'],
            ascending=[True, False]
        )

        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
                tf_idf_df.to_excel(writer, index=False, sheet_name="Query TF-IDF")
            print(f"TF-IDF results (including L2 normalization) saved to '{filename}'.")
        except FileNotFoundError:
            with pd.ExcelWriter(filename, engine="openpyxl") as writer:
                tf_idf_df.to_excel(writer, index=False, sheet_name="Query TF-IDF")
            print(f"File '{filename}' was created and TF-IDF results (including L2 normalization) were saved.")

def select_connected_pages(page_indexes, url_to_id, id_to_url, soup_cache=None, max_pages=50):
    """
    Selects up to `max_pages` pages from the crawled set that are interconnected
    (i.e., have links among themselves), based on the stored soups.

    Args:
        page_indexes (dict): Dictionary of {doc_id: {word: count}}, currently not used
                             but included for future expansion.
        url_to_id (dict): Mapping from URL to its assigned integer ID.
        id_to_url (dict): Mapping from integer ID to the original URL string.
        soup_cache (dict): A dictionary mapping URL -> BeautifulSoup object (for faster access).
        max_pages (int): Maximum number of pages to select.

    Returns:
        dict: A dictionary of {page_id: [list_of_outgoing_page_ids]} for the selected connected pages.
    """
    connected_pages = {}
    page_links = {}

    # Build initial link structure
    for url_id in id_to_url:
        url = id_to_url[url_id]
        soup = soup_cache.get(url) if soup_cache else None
        if soup:
            links = set()
            for link in soup.find_all('a', href=True):
                absolute_url = urljoin(url, link['href'])
                if absolute_url in url_to_id:
                    target_id = url_to_id[absolute_url]
                    links.add(target_id)
            if links:  # Only include pages that have outgoing links
                page_links[url_id] = list(links)

    # Select connected pages (prefer pages with more connections)
    selected_ids = set()
    for page_id, links in sorted(page_links.items(), key=lambda x: len(x[1]), reverse=True):
        if len(selected_ids) >= max_pages:
            break
        connected_links = [l for l in links if l in page_links]
        if connected_links:
            selected_ids.add(page_id)
            selected_ids.update(connected_links[:max_pages-len(selected_ids)])

    # Create final link structure for selected pages
    for page_id in selected_ids:
        outgoing = [l for l in page_links.get(page_id, []) if l in selected_ids]
        if outgoing:
            connected_pages[page_id] = outgoing

    return connected_pages

def calculate_new_pagerank(current_ranks, links, damping_factor=0.85, epsilon=1e-6):
    """
    Computes new PageRank values for one iteration with a damping factor,
    handling dangling nodes, and includes a convergence check.

    Args:
        current_ranks (dict): Current PageRank scores {page_id: score}.
        links (dict): Adjacency list {page_id: [list_of_outgoing_ids]}.
        damping_factor (float): Damping factor for PageRank (usually around 0.85).
        epsilon (float): Threshold for convergence.

    Returns:
        tuple:
            new_ranks (dict): Updated PageRank scores {page_id: score}.
            converged (bool): True if the algorithm has converged under epsilon, else False.
    """
    new_ranks = {}
    num_pages = len(current_ranks)
    dangling_sum = sum(current_ranks[page] for page in current_ranks if page not in links)

    # Iterate over each page to calculate its new rank
    for page in current_ranks:
        rank_sum = 0

        # Find incoming links to the page
        incoming_links = [p for p, outgoing in links.items() if page in outgoing]

        # Sum up contributions from incoming links
        for source_page in incoming_links:
            num_outgoing = len(links[source_page])
            if num_outgoing > 0:  # Avoid division by zero
                rank_sum += current_ranks[source_page] / num_outgoing

        # Add damping factor, dangling nodes contribution, and random jump
        new_ranks[page] = (
            (1 - damping_factor) / num_pages +  # Random jump
            damping_factor * (rank_sum + dangling_sum / num_pages)  # Link contributions
        )

    # Check for convergence
    max_change = max(abs(new_ranks[page] - current_ranks[page]) for page in current_ranks)
    if max_change < epsilon:
        print(f"Converged with max change: {max_change:.6e}")
        return new_ranks, True  # Return new ranks and a flag indicating convergence

    return new_ranks, False  # Return new ranks and a flag indicating non-convergence

def print_ranks(ranks, iteration, id_to_url=None):
    """
    Prints PageRank values after a given iteration.

    Args:
        ranks (dict): Current PageRank scores {page_id: score}.
        iteration (int): The iteration number.
        id_to_url (dict): Mapping from page_id to URL; if provided, the URL is displayed too.

    Returns:
        None
    """
    print(f"\nPageRank values after iteration {iteration}:")
    print("-" * 50)
    if id_to_url:
        print("Page ID | URL | PageRank Value")
    else:
        print("Page | PageRank Value")
    print("-" * 50)
    for page, rank in ranks.items():
        if id_to_url:
            url = id_to_url[page]
            print(f"{page} | {url[:100]}... | {rank:.3f}")
        else:
            print(f"{page} | {rank:.3f}")
    print("-" * 50)

def run_pagerank_analysis(page_indexes, url_to_id, id_to_url, soup_cache, num_iterations=2):
    """
    Executes a full PageRank computation workflow:

    1) Selects connected pages.
    2) Prints the initial PageRank values.
    3) Iteratively computes new PageRank values (up to num_iterations or convergence).
    4) Identifies the page with the highest final PageRank score.

    Args:
        page_indexes (dict): Dictionary of {doc_id: {word: count}} from the crawl.
        url_to_id (dict): Mapping from URL to integer IDs.
        id_to_url (dict): Mapping from integer IDs to URL.
        soup_cache (dict): A cache mapping URL -> BeautifulSoup objects for faster access.
        num_iterations (int): Maximum number of PageRank iterations.

    Returns:
        dict: Final PageRank scores {page_id: score}.
    """

    print("\n=== PageRank Analysis ===")

    # Select connected pages
    connected_pages = select_connected_pages(page_indexes, url_to_id, id_to_url, soup_cache)

    if not connected_pages:
        print("No connected pages found. PageRank analysis cannot proceed.")
        return

    # Show selected pages
    print("\nSelected pages for PageRank analysis:")
    for page_id in connected_pages:
        print(f"\nPage ID: {page_id}, URL: {id_to_url[page_id]}")
        print(f"Links to: {[id_to_url[link_id] for link_id in connected_pages[page_id]]}")

    # Initialize PageRank values
    current_ranks = {page: 1/len(connected_pages) for page in connected_pages}

    # Print initial values
    print("\nInitial PageRank Values:")
    print_ranks(current_ranks, 0, id_to_url)

    # Perform iterations
    for i in range(num_iterations):
        # current_ranks = calculate_new_pagerank(current_ranks, connected_pages)
        # print_ranks(current_ranks, i + 1, id_to_url)
        current_ranks, converged = calculate_new_pagerank(current_ranks, connected_pages)
        print_ranks(current_ranks, i + 1, id_to_url)
        #print(f"Iteration {i+1}: {current_ranks}")
        if converged:
            break

    # Find highest PageRank
    highest_page = max(current_ranks.items(), key=lambda x: x[1])
    print(f"\n\nHighest PageRank after {num_iterations} iterations:")
    print(f"Page ID: {highest_page[0]}")
    print(f"URL: {id_to_url[highest_page[0]]}")
    print(f"PageRank value: {highest_page[1]:.3f}")

    return current_ranks

class FeedbackRelevance:
    def __init__(self, pages, pagerank_scores, id_to_url, search_engine):
        """
        Manages a user feedback system for re-ranking results using relevance feedback.

        Args:
            pages (list): List of page IDs to present to users.
            pagerank_scores (dict): PageRank scores {page_id: score}.
            id_to_url (dict): Mapping from page_id to URL.
            search_engine (SearchEngine): An instance of the SearchEngine class.

            Attributes:
            pages (list): The pages to present for feedback.
            pagerank_scores (dict): Current PageRank scores for each page.
            id_to_url (dict): Map from page_id to URL.
            search_engine (SearchEngine): Reference to the main SearchEngine instance.
            user_feedback (dict): Stores feedback per user (e.g., {user_id: {page_id: bool}}).
            outputs (dict): Stores output widgets per user for displaying status/results.
            button_pairs (dict): Stores pairs of (relevant_button, non_relevant_button) per user.
        """
        self.pages = pages  # List of page IDs
        self.pagerank_scores = pagerank_scores
        self.id_to_url = id_to_url
        self.search_engine = search_engine  # Reference to SearchEngine instance
        self.user_feedback = {}  # Dictionary to store feedback per user
        self.outputs = {}  # Dictionary to store outputs per user
        self.button_pairs = {}  # Dictionary to store button pairs

    def initialize_user(self, user_id):
        """
        Ensures internal data structures are set up for a given user.

        Args:
            user_id (int): Unique identifier for the user.

        Returns:
            None
        """
        if user_id not in self.user_feedback:
            self.user_feedback[user_id] = {}
            self.outputs[user_id] = widgets.Output()
            self.button_pairs[user_id] = {}

    def create_page_display(self, page_id):
            """
            Builds an HTML snippet summarizing a page (URL, headings, etc.) for display.

            Args:
                page_id (int): The unique ID for the page to display.

            Returns:
                str: An HTML string with page information.
            """
            url = self.id_to_url[page_id]

            # Get the page soup from search engine's cache
            soup = self.search_engine.fetch_page(url) if hasattr(self.search_engine, 'fetch_page') else None

            # Extract headings
            headings = []
            if soup:
                # Get main headings (h1, h2)
                for h in soup.find_all(['h1', 'h2'])[:2]:
                    text = h.get_text().strip()
                    if text:
                        headings.append(text)

            # Get the keywords from the search engine's index
            words = self.search_engine.global_index.get(url, {}).keys()
            page_text = " ".join(words)

            return f"""
            <div style="margin: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                <div style="float: right; color: #666;">PageRank Score: {self.pagerank_scores[page_id]:.3f}</div>
                <div style="color: #1a0dab; font-size: 18px;">Page ID: {page_id}</div>
                <div style="color: #006621; font-size: 14px;"><a href="{url}" target="_blank">{url}</a></div>
                <div style="color: #333; font-weight: bold; margin-top: 10px;">
                    {' | '.join(headings) if headings else 'No headings available'}
                </div>
            </div>
            """

    def on_button_click(self, b):
        """
        Handler for marking a document as relevant or non-relevant.
        Updates internal state and changes button styles accordingly.

        Args:
            b (Button): The button that was clicked.
                        It contains 'user_id' and 'page_id' as custom attributes.

        Returns:
            None
        """
        user_id = int(b.user_id)
        page_id = int(b.page_id)
        is_relevant = b.description.startswith('👍')

        # Update feedback
        self.user_feedback[user_id][page_id] = is_relevant

        # Update button styles
        rel_btn, nonrel_btn = self.button_pairs[user_id][page_id]
        if is_relevant:
            rel_btn.button_style = 'success'
            nonrel_btn.button_style = ''
        else:
            rel_btn.button_style = ''
            nonrel_btn.button_style = 'danger'

        # Show current feedback status
        with self.outputs[user_id]:
            clear_output()
            print(f"✓ Marked document {page_id} as {'relevant' if is_relevant else 'not relevant'}")
            if self.user_feedback[user_id]:
                print("\nCurrent feedback:")
                for pid, v in sorted(self.user_feedback[user_id].items()):
                    status = "relevant" if v else "not relevant"
                    print(f"Document {pid}: {status}")

    def rerank_results(self, user_id):
        """
        Re-ranks the pages using Rocchio-like relevance feedback.
        Creates TF-IDF vectors for all pages, then adjusts the query vector
        based on user feedback and computes new rankings via cosine similarity.

        Args:
            user_id (int): Identifier for the user whose feedback should be used.

        Returns:
            tuple:
                reranked_pages (list): A list of page IDs in descending order of new scores.
                scores (numpy.ndarray): Corresponding similarity scores for each page.
        """
        relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == True]
        non_relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == False]

        # Create TF-IDF vectors
        all_words = set()
        page_word_counts = {}

        # Calculate document frequencies (DF)
        doc_freq = defaultdict(int)
        for page_id in self.pages:
            word_counts = {word: counts[page_id] for word, counts in self.search_engine.inverted_index.items()
                        if page_id in counts}
            for word in word_counts:
                doc_freq[word] += 1
            all_words.update(word_counts.keys())
            page_word_counts[page_id] = word_counts

        # Calculate total words per document
        doc_total_words = {}
        for doc_id, word_counts in page_word_counts.items():
            doc_total_words[doc_id] = sum(word_counts.values())

        # Create word-to-index mapping
        word_to_idx = {word: idx for idx, word in enumerate(sorted(all_words))}

        # Create TF-IDF vectors
        doc_vectors = []
        N = len(self.pages)  # Total number of documents

        for page_id in self.pages:
            vector = np.zeros(len(all_words))
            word_counts = page_word_counts[page_id]
            total_words = doc_total_words[page_id]

            for word, count in word_counts.items():
                idx = word_to_idx[word]
                tf = count / total_words if total_words > 0 else 0
                idf = math.log(N / doc_freq[word]) if doc_freq[word] > 0 else 0
                vector[idx] = tf * idf

            doc_vectors.append(vector)

        doc_vectors = np.array(doc_vectors)

        # Initialize query vector as mean of all documents
        #To get an initial query vector, we take the average of all the document vectors.
        query_vector = np.mean(doc_vectors, axis=0)

        # Modify query vector based on feedback
        beta = 0.75  # Weight for relevant documents
        gamma = 0.15  # Weight for non-relevant documents

        if relevant_pages:
            relevant_indices = [self.pages.index(page_id) for page_id in relevant_pages]
            query_vector += beta * np.mean(doc_vectors[relevant_indices], axis=0)

        if non_relevant_pages:
            non_relevant_indices = [self.pages.index(page_id) for page_id in non_relevant_pages]
            query_vector -= gamma * np.mean(doc_vectors[non_relevant_indices], axis=0)

        # Calculate similarities using sklearn's cosine_similarity
        similarities = cosine_similarity(doc_vectors, query_vector.reshape(1, -1)).flatten()
        ranked_indices = np.argsort(-similarities)

        return [self.pages[i] for i in ranked_indices], similarities[ranked_indices]

    def show_ranking(self, b):
        """
        Displays a new ranking based on user feedback.
        Triggered by "Show Ranking" button click.

        Args:
            b (Button): The button widget that triggered this method.

        Returns:
            None
        """
        user_id = int(b.user_id)

        if not self.user_feedback[user_id]:
            with self.outputs[user_id]:
                clear_output()
                print("Please mark at least one document as relevant or not relevant.")
                return

        reranked_pages, scores = self.rerank_results(user_id)

        with self.outputs[user_id]:
            clear_output()
            print(f"🔄 Reranked results for User {user_id}:\n")
            for page_id, score in zip(reranked_pages, scores):
                display(HTML(f"""
                    <div style="margin: 20px; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                        <div style="float: right; color: #666;">
                            New Score: {score:.3f} | Original PageRank: {self.pagerank_scores[page_id]:.3f}
                        </div>
                        <div style="color: #1a0dab; font-size: 18px;">Page ID: {page_id}</div>
                        <div style="color: #006621; font-size: 14px;">{self.id_to_url[page_id]}</div>
                        <div style="color: #545454; margin-top: 5px;">
                            Feedback: {
                                "Relevant ✓" if self.user_feedback[user_id].get(page_id) == True
                                else "Not Relevant ✗" if self.user_feedback[user_id].get(page_id) == False
                                else "No Feedback"
                            }
                        </div>
                    </div>
                """))

    def display_interface(self, user_id):
        """
        Builds and displays the interactive feedback interface (page listings,
        relevant/not relevant buttons, and ranking buttons) for a specific user.

        Args:
            user_id (int): Unique identifier for the user.

        Returns:
            None
        """
        self.initialize_user(user_id)

        # Create header
        display(HTML(f"""
            <div style="margin: 20px 0; padding: 10px; background-color: #f8f9fa; border-radius: 5px;">
                <h3 style="margin: 0; color: #333;">Feedback Interface for User {user_id}</h3>
                <p style="margin: 10px 0 0 0; color: #666;">
                    Click 👍 or 👎 to mark pages as relevant or not relevant, then use the buttons below
                    to see reordered results or get query suggestions.
                </p>
            </div>
        """))

        for page_id in self.pages:
            display(HTML(self.create_page_display(page_id)))

            # Create buttons
            rel_btn = widgets.Button(
                description='👍 Relevant',
                tooltip=str(page_id),
                layout=widgets.Layout(margin='5px')
            )
            nonrel_btn = widgets.Button(
                description='👎 Not Relevant',
                tooltip=str(page_id),
                layout=widgets.Layout(margin='5px')
            )

            # Add user_id and page_id as attributes
            rel_btn.user_id = str(user_id)
            rel_btn.page_id = str(page_id)
            nonrel_btn.user_id = str(user_id)
            nonrel_btn.page_id = str(page_id)

            # Store button pair
            self.button_pairs[user_id][page_id] = (rel_btn, nonrel_btn)

            # Set up click handlers
            rel_btn.on_click(self.on_button_click)
            nonrel_btn.on_click(self.on_button_click)

            display(widgets.HBox([rel_btn, nonrel_btn]))

        # Create action buttons container
        button_container = widgets.HBox([
            widgets.Button(
                description='Show Ranking',
                button_style='primary',
                layout=widgets.Layout(margin='20px 10px 20px 0')
            ),
            widgets.Button(
                description='Show Query Suggestion',
                button_style='info',
                layout=widgets.Layout(margin='20px 0 20px 10px')
            )
        ])

        # Set up button attributes and handlers
        rank_button = button_container.children[0]
        suggest_button = button_container.children[1]

        rank_button.user_id = str(user_id)
        suggest_button.user_id = str(user_id)

        rank_button.on_click(self.show_ranking)
        suggest_button.on_click(self.display_query_suggestion)

        display(button_container)
        display(self.outputs[user_id])

    def suggest_expanded_query(self, user_id):
        """
        Generates additional query terms using relevance feedback from the user.
        Calculates term weights, and selects top terms that appear in relevant documents
        but are less frequent in non-relevant ones.

        Args:
            user_id (int): Unique identifier for the user.

        Returns:
            tuple:
                expanded_query (str or None): Suggested query terms joined by spaces,
                                             or None if no suggestion is possible.
                explanations (str or None): Explanation of chosen terms, or None if not applicable.
        """
        # Get relevant and non-relevant documents
        relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == True]
        non_relevant_pages = [i for i, v in self.user_feedback[user_id].items() if v == False]

        if not relevant_pages:
            return None, None

        # Collect terms from relevant documents
        term_weights = defaultdict(float)
        term_counts = defaultdict(int)

        # Calculate term frequencies in relevant documents
        for page_id in relevant_pages:
            url = self.id_to_url[page_id]
            # Get terms from the inverted index
            for term, doc_counts in self.search_engine.inverted_index.items():
                if page_id in doc_counts:
                    term_weights[term] += doc_counts[page_id]
                    term_counts[term] += 1

        # Penalize terms that appear in non-relevant documents
        for page_id in non_relevant_pages:
            url = self.id_to_url[page_id]
            for term, doc_counts in self.search_engine.inverted_index.items():
                if page_id in doc_counts:
                    term_weights[term] -= doc_counts[page_id] * 0.5  # Penalty factor

        # Calculate final term weights using Rocchio-like approach
        alpha = 1.0  # Original query weight
        beta = 0.75  # Relevant documents weight
        gamma = 0.15  # Non-relevant documents weight

        # Sort terms by weight and get top N terms
        N = 5  # Number of terms to suggest
        suggested_terms = sorted(
            [(term, weight) for term, weight in term_weights.items()],
            key=lambda x: x[1],
            reverse=True
        )[:N]

        # Create expanded query suggestion
        expansion_terms = []
        term_explanations = []

        for term, weight in suggested_terms:
            if weight > 0:
                expansion_terms.append(term)
                explanation = (
                    f"'{term}' (weight: {weight:.2f}, "
                    f"appears in {term_counts[term]} relevant documents)"
                )
                term_explanations.append(explanation)

        if not expansion_terms:
            return None, None

        expanded_query = " ".join(expansion_terms)
        explanations = "\n".join(f"- {exp}" for exp in term_explanations)

        return expanded_query, explanations

    def display_query_suggestion(self, b):
        """
        Button callback that displays a query expansion suggestion to the user.

        Args:
            b (Button): The button widget that triggered this method.

        Returns:
            None
        """
        user_id = int(b.user_id)

        if not self.user_feedback[user_id]:
            with self.outputs[user_id]:
                clear_output()
                print("Please provide feedback on some documents first.")
                return

        expanded_query, explanations = self.suggest_expanded_query(user_id)

        with self.outputs[user_id]:
            clear_output()
            if expanded_query:
                print("📝 Suggested Query Expansion:")
                print(f"\nExpanded Query: {expanded_query}")
                print("\nTerm Selection Explanation:")
                print(explanations)
                print("\nThis suggestion is based on the terms that appear frequently in")
                print("documents you marked as relevant, while avoiding terms from")
                print("non-relevant documents.")
            else:
                print("Unable to generate query suggestion.")
                print("Please mark more documents as relevant/non-relevant.")

    def add_suggestion_button(self):
        """
        (Optional) Adds a standalone button to show query suggestions.
        Can be used if we want to manually place a separate suggestion button.

        Returns:
            None
        """
        suggest_button = widgets.Button(
            description='Show Query Suggestion',
            button_style='info',
            layout=widgets.Layout(margin='20px 0')
        )
        suggest_button.user_id = str(user_id)
        suggest_button.on_click(self.display_query_suggestion)
        display(suggest_button)

def implement_feedback_relevance(pagerank_scores, id_to_url, search_engine):
    """
    Builds and displays the feedback interface for multiple users
    to provide relevance feedback on the pages.

    Args:
        pagerank_scores (dict): PageRank scores {page_id: score}.
        id_to_url (dict): Mapping from page_id to URL.
        search_engine (SearchEngine): Reference to the SearchEngine instance.

    Returns:
        FeedbackRelevance: The feedback system object to allow further interaction.
    """
    # מיון הדפים לפי ציון PageRank וקבלת 20 המובילים
    top_20_pages = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)[:20]

    # יצירת רשימת ה-IDs של 20 הדפים המובילים
    top_pages = [page_id for page_id, score in top_20_pages]

    # יצירת מילון חדש עם הציונים רק של 20 הדפים הנבחרים
    filtered_scores = {page_id: score for page_id, score in top_20_pages}

     # יצירת מערכת המשוב עם הדפים המסוננים
    feedback_system = FeedbackRelevance(top_pages, filtered_scores, id_to_url, search_engine)

    # הצגת הממשק לשני המשתמשים
    print("\nDisplaying Feedback Interface for User 1...")
    feedback_system.display_interface(1)

    print("\nDisplaying Feedback Interface for User 2...")
    feedback_system.display_interface(2)

    return feedback_system

def main():
    """
    Main driver function with integrated crawling and PageRank analysis
    Main driver function that:
    1) Initializes the SearchEngine.
    2) Performs crawling, indexing, and data caching.
    3) Analyzes link structure (HITS).
    4) Runs PageRank analysis.
    5) Saves initial results to Excel.
    6) Calculates TF-IDF for a sample query and appends it to an Excel file.
    7) Initializes and displays a user feedback relevance system.
    """
    search_engine = SearchEngine()
    start_url = "https://www.bundesgesundheitsministerium.de/en/"
    query = "How can I replace my lost European Health Insurance Card (EHIC)?"

    print("Starting crawling and indexing...")
    start_crawl = time.time()

    # Use crawl_and_index for both indexing and PageRank
    page_indexes, soup_cache = search_engine.crawl_and_index(start_url, query,max_pages=51)

    end_crawl = time.time()
    crawl_time = end_crawl - start_crawl
    print(f"Crawling and indexing completed in {crawl_time:.2f} seconds.")

    print(f"Processing query: {query}")
    start_tfidf = time.time()
    filename = f"results_query_{query}.xlsx"
    search_engine.calculate_tf_idf_for_query(query, page_indexes, filename)
    end_tfidf = time.time()
    tfidf_time = end_tfidf - start_tfidf

    with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        timing_data = pd.DataFrame([{
            "Crawling Time (seconds)": crawl_time,
            "TF-IDF Calculation Time (seconds)": tfidf_time
        }])
        timing_data.to_excel(writer, index=False, sheet_name="Timing Information")

    print(f"TF-IDF calculation completed in {tfidf_time:.2f} seconds.")
    print(f"Results saved to '{filename}'.")

    # Save results and continue with existing functionality
    search_engine.save_to_excel(page_indexes, "initial_results.xlsx")

    # Run HITS Algorithm Analysis
    link_analysis = search_engine.analyze_link_structure(max_iterations=20, epsilon=1e-8)

    # Run PageRank analysis with the soup cache
    pagerank_scores = run_pagerank_analysis(
        page_indexes,
        search_engine.url_to_id,
        search_engine.id_to_url,
        soup_cache,
        num_iterations=2
    )

    # Initialize feedback system
    print("\nInitializing Feedback Relevance System...")
    feedback_system = implement_feedback_relevance(pagerank_scores, search_engine.id_to_url, search_engine)

if __name__ == "__main__":
    main()



Starting crawling and indexing...
Crawling: https://www.bundesgesundheitsministerium.de/en/international/global-health-policy.html (ID: 1)
Crawling: https://www.bundesgesundheitsministerium.de/en/themen/praevention/antibiotika-resistenzen/dart-2030.html (ID: 2)
Crawling: https://www.bundesgesundheitsministerium.de/en/topics/digitalisation/digitalisation-strategy.html (ID: 3)
Crawling: https://www.bundesgesundheitsministerium.de/en/en/international/global-challenges.html (ID: 4)
Crawling: https://www.bundesgesundheitsministerium.de/en/topics/digitalisation/digitalisation-strategy/the-strategy-development-process.html (ID: 5)
Crawling: https://www.bundesgesundheitsministerium.de/en/topics/antimicrobial-resistance.html (ID: 6)
Crawling: https://www.bundesgesundheitsministerium.de/en/data-protection.html (ID: 7)
Crawling: https://www.bundesgesundheitsministerium.de/en/themen/cannabis/faq-cannabis-act.html (ID: 8)
Crawling: https://www.bundesgesundheitsministerium.de/en/ministry/leadership/

Output()


Displaying Feedback Interface for User 2...


Output()